In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from functools import partial
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit
from sklearn.base import clone
from ISLP.models import sklearn_sm

The Validation Set Approach

In [2]:
# The usual way

Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=22)

In [3]:
# Now we can fit simple linear regression iusing onlt the observations corresponding to the training set Auto_train.
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [4]:
# We now use the predict() method of results evaluated on the model matrix for this model created using the validation data set. We also calculate the validation MSE of our model

X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2 )

24.483107068866275

In [7]:
#  Hence our estimate for the validation MSE of the linear regression fit is 24.4831. We can also estimate teh validation error for higher-degree polynomial regressions.WE first provide a funciton evalMSE() that take a model string as well as a training and test set and return the MSE on the test set.

def evalMSE(terms, response, train, test):
        mm = MS(terms)
        X_train = mm.fit_transform(train)
        y_train = train[response]

        X_test = mm.transform(test)
        y_test = test[response]

        results = sm.OLS(y_train, X_train).fit()
        test_pred = results.predict(X_test)

        return np.mean((y_test - test_pred)**2)

In [8]:
# Lets use this function to estimate the validation MSE using linear, quadrativ and cubif fits. WE use the enumerate() funciton here, which gives both the values and indices of objects as one iterates over a for loop.

MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([24.48310707, 19.28199978, 19.25054563])

In [12]:
# So above are the MSE s measured for different degrees
# However if we choose a different training/validation split instead then we can expect somewhwt different errors on the validation set

Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=23)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([22.98621247, 19.03806536, 19.5506616 ])

In [13]:
# The above results are quite consistent with the previous ones



Cross-Validation

In [15]:
# In theory, the cross-validation estimate can be computed for any generalized linear model. In practice, however the simplest way to cross-validate in Python is to use sklearn, which has a different interface or API than statsmodels, the code we have been using to fit GLMs.

# This is a problem which often confronts data scientists: "I have a function to do task A, and need to feed it into something that performs task B, so that I can compute B(A(D)) wehre D is my data." When A and B dont naturally speak to each other, this requires the use of a wrapper. 

sklearn_sm?

Init signature: sklearn_sm(model_type, model_spec=None, model_args={})
Docstring:     
Parameters
----------

model_type: class
    A model type from statsmodels, e.g. sm.OLS or sm.GLM

model_spec: ModelSpec
    Specify the design matrix.

model_args: dict (optional)
    Arguments passed to the statsmodels model.

Notes
-----

If model_str is present, then X and Y are presumed
to be pandas objects that are placed
into a dataframe before formula is evaluated.
This affects `fit` and `predict` methods.
File:           ~/Macos/py3.11/lib/python3.11/site-packages/ISLP/models/sklearn_wrap.py
Type:           type
Subclasses:     sklearn_selected, sklearn_selection_path

In [17]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])


cv_err = np.mean(cv_results['test_score'])
cv_err  

24.23151351792922

In [18]:
# The arguments to cross_validate() are as follows: an object witht the appropriate fit(), predict() and score() methods an array of features X and response Y. We also included an additional argument cv to cross_validate(); specifying an integer K results in K-fold cross-validation. We have provided a value correstponding to the total number of observations, which results in leave-one-out cross-validation(LOOCV). The cross_validate() function produces a dictionary with several components; we simply want the cross-validated test score here (MSE), which is estimated to be 24.23

# We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 5, computes the associated cross-validation error, and stores it in the ith element of the vector cv_error. The variable d in the for loop corresponds to the degree of the polynomial. We begin by initializing the vector. This command may take a couple of seconds to run



cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M  = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,X,Y,cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

In [19]:
# As in figure we see a sharp drop in the estimated test MSE between the linear quadratic fits, but then no clear imporvement from using higher-degree polynomials. Above we introduced the outer() method of the np.power() function. The outer() mehtod is applied to an operation that has two arguments,

# The np.add.outer function in numpy computes the element-wise addition of two arrays. It takes two arrays, x and y as input and re
A = np.array([3,5,9])
B = np.array([2,4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [24]:
# In the CV example above, we used K = n, but of course we can also use K<n. The code is very similar to the above. Here we use KFold() to partition the data into K=10 random groups. We use random_state to set a random seed and initialize a vector cv_error in which we will store the CV errors corresponding to the polynomial fits of degrees one to five


cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720065])

In [29]:
# Notice that the computation time is much shorter than that of LOOCV. (In principle the computation time for LOOCV for a least squares linear model should be faster that for K-fold CV, due to the availability of the formula for LOOCV; however the generic cross_validate() function does not make use of this formula) We still see little evidence that using cubic or higher-degree polynomial terms leads to a lower test error than simply using a quadratic fit.


validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)

results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)

results['test_score']

array([23.617])

## The Bootstrap

In [30]:
# We illustrate the use of the bootstrap in the simple example.As well as on an example involving estimating the accuracy of the linear regression model on the Auto data set

In [31]:
# Estimating the Accuracy of a Statistic of Interest

# One of the great advantages of the bootstrap approach is that it can be applied in almost all situations. No complicated  mathematical calculations are required. While there are several implementations of the bootstrap in Python, its use for estimating standard error is simple enought that we write out own funciton below for the case when our data is stored in a dataframe.


Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1])/(cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [32]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [33]:
# Next we randomly select 100 observations from range(100), with replacement. This is equivalent to constructing a new bootstrap data set and recomputing alpha based on the new data set


rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

In [34]:
#Next we randomly select 100 observations from range(100), with replacement. This is equivalent to constructing a new bootstrap data set and recomputing alpha based on the new data set


def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n for D.shape[0]
    

0.6074452469619004

In [35]:
# This preocess canbe generalized to create a simple function boot_SE() for computing the bootsrap standard error for arbitrary functions that take only a data frame as an argument.


def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_/B - (first_/B)**2)

In [37]:
# Notice the use of _ as a loop variable in for _ in range(B). This is often used if the value of the counter is unimportant and simply makes suere the loop is executed B times.


# Lets use our function to evaluate the accuracy of our estimate of alpha using B=1000

alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)

alpha_SE

0.09118176521277699

In [38]:
# The final output shows that the bootstrap estimate for SE(alpha) is 0.0912



Estimating the Accuracy of a Linear Regression Model

In [39]:
# We start by writing a generic function boot_OLS() for bootstrapping a regression model that takes a formula to define the corresponding regression. We use the clone() funciton to make a copy of the formula that can be refit to the new datafram. This means that any derived deatures such as those defined by poly(), will be re-fit on the resampled data frame.

def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [41]:
# This is not quite what is needed as the first argument to boot_SE(). The first two arguments which specify the model will not change in the bootstrap process, and we would like to freeze them. The function partial() from the functools module does precisely this: it takes a function as an argument, and freezes some of its arguments, starting from the left. We use it to freeze the first two model-formula arguments of boot_OLS()

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x7f03dffa2de0>, ModelSpec(terms=['horsepower']), 'mpg')
File:           ~/Macos/py3.11/lib/python3.11/functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [42]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

array([[39.881, -0.157],
       [38.733, -0.147],
       [38.317, -0.144],
       [39.914, -0.158],
       [39.433, -0.151],
       [40.366, -0.159],
       [39.623, -0.154],
       [39.058, -0.15 ],
       [38.667, -0.145],
       [39.643, -0.156]])

In [43]:

# Next we use the boot_SE() funciton to compute the standard errors of 1000 estimates

hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

In [44]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [45]:
# we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model to the data. Since this model porvides a good fit to the data


quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [46]:
# We compare the results to the standard errors computed using sm.OLS()

M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64